In [1]:
#import necessary packages
import pandas as pd
import numpy as np
import os
from urllib.request import urlopen
import json
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from scipy import interpolate
from scipy.interpolate import griddata
import pyepsg

Using the GBIF API to download iNaturalist data

Jillian: Time range / time series of species

Erica: spatial distribution of a species 
Kiran: Synthesizing iNaturalist data (counts of observations?)


In [1]:
import pygbif
#need to run pip install pygbif in terminal

ModuleNotFoundError: No module named 'pygbif'

In [ ]:
documentaton

doc

Using iNaturalist Research - Grade Observations
https://www.gbif.org/dataset/50c9509d-22c7-4a22-a47d-8c48425ef4a7

Help for using the API: https://www.gbif.org/developer/summary



In [2]:
# Read in your token
#iucn_token="9bb4facb6d23f48efbf424bb05c0c1ef1cf6f468393bc745d42179ac4aca5fee"

# Root URL for GBIF API
gbif_url=" https://api.gbif.org/v1/"

#don't need a token, just your username and password for your GBIF account